Data Extraction and Transformation

One challenge we have at Stadium Goods is correctly reporting international orders when sales are made in different currencies. To solve this, we pull reference exchange rate data that can be used to calculate and report international sales in different currencies. The European Central Bank provides an API to collect this data, documentation can be found here - the data tab should provide the information needed for the excercise.

Using python, write a script that will pull exchange rate data that can be used for reporting. For this exercise, we are looking for data for February 9-10, 2023 using the Key Euro Area Indicators dataflow. Pull data for all daily currencies against the Euro.

Using fake ecommerce data using the API documented here - pull data for all products in the ‘Shoes’ category. Assuming the data for these products is in USD, create a new pandas dataframe that includes all of the products, the price in USD, the price in EUR, and the date of the exchange rate that was used. 

In [379]:
import requests
import pandas as pd
import numpy as np
import json
import requests
import urllib.request
import csv
import io
from datetime import datetime
import os
import subprocess
import logging

# set the API endpoint URL to get data from api

#Get all products by category shoes

url = "https://api.escuelajs.co/api/v1/categories/4/products"
url_data = json.load(urllib.request.urlopen(url))
shoes=pd.json_normalize(url_data)
print(shoes.tail())
products=shoes[['creationAt','id','category.name','title','price']]
#Schema created

     id                      title  price  \
27  163   Intelligent Bronze Pants     45   
28  172   Incredible Bronze Cheese    614   
29  177  Luxurious Cotton Computer    517   
30  190         Small Frozen Bacon    218   
31  200       Refined Rubber Pizza    938   

                                          description  \
27  The Apollotech B340 is an affordable wireless ...   
28  The Football Is Good For Training And Recreati...   
29  Ergonomic executive chair upholstered in bonde...   
30  Ergonomic executive chair upholstered in bonde...   
31                     Boston's most advanced compres   

                                               images  \
27  [https://api.lorem.space/image/shoes?w=640&h=4...   
28  [https://api.lorem.space/image/shoes?w=640&h=4...   
29  [https://api.lorem.space/image/shoes?w=640&h=4...   
30  [https://api.lorem.space/image/shoes?w=640&h=4...   
31  [https://api.lorem.space/image/shoes?w=640&h=4...   

                  creationAt               

In [380]:
eurobank='https://sdw-wsrest.ecb.europa.eu/service/data/EXR/M.USD+GBP+JPY.EUR.SP00.A?startPeriod=2023-02-01'
headers = {'Accept':'text/csv'}
params = {'startPeriod': '2023-02-09', 'endPeriod': '2023-02-10'}

# send a GET request to the API endpoint ECB SDMX 2.1 RESTful web service
response = requests.get(eurobank, headers=headers)
response

#check the response status code (200 means success)
if response.status_code == 200:
    data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(data))
    print(df.head())
    # Parse the CSV data
    #reader = csv.DictReader(data.splitlines())
    #rows = [row for row in reader]
    # Process the data
    #for row in rows:
        #print(row)
else:
    print('Error retrieving data:', response.text)

                    KEY FREQ CURRENCY CURRENCY_DENOM EXR_TYPE EXR_SUFFIX  \
0  EXR.M.GBP.EUR.SP00.A    M      GBP            EUR     SP00          A   
1  EXR.M.JPY.EUR.SP00.A    M      JPY            EUR     SP00          A   
2  EXR.M.USD.EUR.SP00.A    M      USD            EUR     SP00          A   

  TIME_PERIOD  OBS_VALUE OBS_STATUS OBS_CONF  ...  COMPILATION  COVERAGE  \
0     2023-02    0.88550          A        F  ...          NaN       NaN   
1     2023-02  142.37700          A        F  ...          NaN       NaN   
2     2023-02    1.07151          A        F  ...          NaN       NaN   

  DECIMALS  NAT_TITLE SOURCE_AGENCY  SOURCE_PUB                   TITLE  \
0        5        NaN           4F0         NaN  UK pound sterling/Euro   
1        2        NaN           4F0         NaN       Japanese yen/Euro   
2        4        NaN           4F0         NaN          US dollar/Euro   

                                         TITLE_COMPL  UNIT  UNIT_MULT  
0  ECB reference 

In [381]:
#Creating database and applying  the rules to solve the case
df1=df[['CURRENCY','OBS_VALUE','TIME_PERIOD']].copy()
df1.sort_values(['TIME_PERIOD'])
products['CURRENCY']='USD'
database=pd.merge(products,df1,how='left',on='CURRENCY')
database['price_euro']=round(database.price*database.OBS_VALUE,2)
database=database[['category.name','price','price_euro','TIME_PERIOD']]
database.rename(columns={'price':'price_usd', 'TIME_PERIOD':'date_exchange'},inplace=True)
database['date']=datetime.now()

C:\Users\LUCAS BARBOSA\AppData\Local\Temp\ipykernel_17752\1474307460.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['CURRENCY']='USD'


In [382]:
database

,category.name,price_usd,price_euro,date_exchange,date
0,Shoes,464,497.18,2023-02,2023-03-09 21:20:12.670112
1,Shoes,467,500.40,2023-02,2023-03-09 21:20:12.670112
2,Shoes,698,747.91,2023-02,2023-03-09 21:20:12.670112
3,Shoes,649,695.41,2023-02,2023-03-09 21:20:12.670112
4,Shoes,601,643.98,2023-02,2023-03-09 21:20:12.670112
5,Shoes,518,555.04,2023-02,2023-03-09 21:20:12.670112
6,Shoes,611,654.69,2023-02,2023-03-09 21:20:12.670112
7,Shoes,760,814.35,2023-02,2023-03-09 21:20:12.670112
8,Shoes,982,1052.22,2023-02,2023-03-09 21:20:12.670112
9,Shoes,926,992.22,2023-02,2023-03-09 21:20:12.670112


In [ ]:
database.to_csv('database.csv')

Data Engineering in Production

For these questions please provide a brief (1 paragraph) explanation. Diagrams can be included if it is helpful, but not required.

Using typical sales data as an example, how would you ensure that a data pipeline is kept up to date with accurate data? What tools or process might you use so that sales data is updated daily?

1)First of all apply some data quality process as duplicate entries, missing data. Checking for data consistency between different data sources. 
For instance, if you have sales data that is collected from multiple sources you  want to ensure that the data from each source is consistent and accurate.
Regularly review and optimize the data pipeline to ensure that it is efficient and effective.
Logging and sendry_sdk can help.

2)Our sales and product data is constantly changing - returns can affect previous sales, pricing changes can affect product data tables, etc. - how would you go about building a data pipeline that is able to add new data while also changing or updating existing data that has changed at the source system?

In this case is high recomendly Transform and load the data into a  system that can store and manage the data. This can involve cleaning the data, aggregating it, or joining.
Thus, we can monitor the pipeline regularly to ensure that it is running smoothly and effectively.
This can involve setting up alerts for data quality issues, reviewing performance metrics, or optimizing the pipeline for better efficiency.
A example below:

In [ ]:
#2)How would you go about building a data pipeline that is able to add new data while also changing or updating
#existing data that has changed at the  process?
existing_data = pd.read_csv('database.csv')


# Filter data that already exists in target system
new_data = pd.merge(existing_data, on=['date'], how='left', indicator=True)


# Append new data to existing data
all_data = pd.concat([existing_data, new_data])

# Save data
all_data.to_csv('database.csv', index=False)


# Configure logging
logging.basicConfig(filename='database.log', level=logging.INFO)

# Configure Sentry
sentry_sdk.init(dsn=dsn)

# Log errors and exceptions
try:
    # Run data pipeline code here
except Exception as e:
    logging.exception(e)
    sentry_sdk.capture_exception(e)